In [1]:
import pandas as pd
import numpy as np
from lxml import etree
import requests
import urllib3


#### We get the table from the Wikipedia link


In [2]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
html = requests.get(url).content
df_list=pd.read_html(html)

df = df_list[0]
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


#### With this piece of code we can drop the rows where the column "Borough" has a "Not assigned" value, and set the rows with a "Not assigned" value in the "Neighbourhood" column as equal to the "Borough" name.
#### We also reset the index so it starts in 0.

In [3]:
index_list=[]
for i in df.index:
    if df.iloc[i]["Borough"]=="Not assigned":
        index_list.append(i)
    elif df.iloc[i]["Neighbourhood"]=="Not assigned":
        df.iloc[i]["Neighbourhood"]=df.iloc[i]["Borough"]
        
new_df=df.drop(df.index[index_list])
new_df=new_df.reset_index(drop=True)
new_df.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


#### We clean the dataframe by grouping our data by Postcode, and then joining the Neighbourhood data in one string

In [4]:
clean_df=new_df.groupby(["Postcode","Borough"], as_index=False)["Neighbourhood"].agg(lambda col: ', '.join(col))
clean_df.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


#### We load the coordinates

In [5]:
coord = pd.read_csv('Geospatial_Coordinates.csv')
coord.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


#### We check if it has the same shape as our clean dataframe

In [6]:
coord.shape

(103, 3)

#### We join both dataframes, dropping the "Postal Code" column from the csv file so we don't have repeated columns in our new dataframe

In [7]:
geo_df = clean_df.join(coord).drop(["Postal Code"], axis=1)
geo_df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
